## 題目

1. Use LSTM & CNN model to classify MNIST dataset with at least 90%
2. Use LSTM & CNN model to classify customized candlestick pattern (at least 3 classes)

### 執行

所有檔案: mnist_train_all.py、 candlestick_train_cnn.py、candlestick_train_lstm.py

#### 1. Use LSTM & CNN model to classify MNIST
* mnist_model.py

In [1]:
from sklearn.metrics import confusion_matrix
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam


def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

def cnn_preprocess(x_train, x_test, y_train, y_test):
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return (x_train, x_test, y_train, y_test)

def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model

def cnn_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPool2D(strides=2))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(MaxPool2D(strides=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

def trainning(model, x_train, y_train, x_test, y_test, 
              learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
              batch_size=batch_size, epochs=training_iters,
              verbose=1, validation_data=(x_test, y_test))

def print_confusion_result(x_train, x_test, y_train, y_test, model):
    # get train & test predictions
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    
    # get train & test true labels
    train_label = y_train
    test_label =  y_test
    
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(10))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(10))
    print(train_result_cm, '\n'*2, test_result_cm)

def mnist_lstm_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 1
    batch_size = 128

    # model parameters
    n_input = 28
    n_step = 28
    n_hidden = 256
    n_classes = 10

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test, y_train_o, y_test_o = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    trainning(model, x_train, y_train_o, x_test, y_test_o, learning_rate, training_iters, batch_size)
    scores = model.evaluate(x_test, y_test_o, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print_confusion_result(x_train, x_test, y_train, y_test, model)

def mnist_cnn_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 1
    batch_size = 64

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test, y_train_o, y_test_o = cnn_preprocess(x_train, x_test, y_train, y_test)

    model = cnn_model()
    trainning(model, x_train, y_train_o, x_test, y_test_o, learning_rate, training_iters, batch_size)
    scores = model.evaluate(x_test, y_test_o, verbose=0)
    print('CNN test accuracy:', scores[1])
    print_confusion_result(x_train, x_test, y_train, y_test, model)

- LSTM

In [2]:
mnist_lstm_main()

11493376/11490434 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               291840    
_________________________________________________________________
dense (Dense)                (None, 10)                2570      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 294,410
Trainable params: 294,410
Non-trainable params: 0
_________________________________________________________________
469/469 [==============================] - 77s 165ms/step - loss: 0.4429 - accuracy: 0.8536 - val_loss: 0.1551 - val_accuracy: 0.9533
LSTM test accuracy: 0.9532999992370605
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g

- CNN

In [3]:
mnist_cnn_main()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 48)        38448     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 48)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               307456    
_________________________________________________________________
dense_2 (Dense)              (None, 84)               

#### 2. Use LSTM model to classify customized candlestick pattern
* candlestick_model_lstm.py

In [20]:
from sklearn.metrics import confusion_matrix
import pickle
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    # Add dropout layer to prevent overfitting due to more iter
    #model.add(LSTM(n_hidden, dropout=0.2, recurrent_dropout=0.2, batch_input_shape=(None, n_step, n_input), unroll=True))
    # No dropout version
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model

def train_lstm(model, x_train, y_train, x_test, y_test, 
        learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam,
        loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
        batch_size=batch_size, epochs=training_iters,
        verbose=1, validation_data=(x_test, y_test))

def print_result(data, x_train, x_test, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

def lstm_main(training_iters):
    # training parameters
    learning_rate = 0.001
    training_iters = training_iters
    batch_size = 128

    # model parameters
    n_input = 40
    n_step = 10
    n_hidden = 256
    n_classes = 10

    data = load_pkl('/content/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl')
    x_train, y_train, x_test, y_test = data['train_gaf'], data['train_label'][:, 0], data['test_gaf'], data['test_label'][:, 0]
    x_train, x_test, y_train, y_test = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    train_lstm(model, x_train, y_train, x_test, y_test, learning_rate, 
               training_iters, batch_size)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print_result(data, x_train, x_test, model)

In [18]:
# No dropout, 10 iter
lstm_main(10)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 256)               304128    
_________________________________________________________________
dense_12 (Dense)             (None, 10)                2570      
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
Total params: 306,698
Trainable params: 306,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
118/118 [==============================] - 8s 64ms/step - loss: 2.0537 - accuracy: 0.2178 - val_loss: 1.7171 - val_accuracy: 0.2528
Epoch 2/10
118/118 [==============================] - 7s 62ms/step - loss: 1.4472 - accuracy: 0.3566 - val_loss: 1.3081 - val_accuracy: 0.4096
Epoch 3/10
118/118 [==============================] - 7s 61ms/step - loss: 1.3148 - accuracy

In [19]:
# No dropout, 50 iter
lstm_main(50)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 256)               304128    
_________________________________________________________________
dense_13 (Dense)             (None, 10)                2570      
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
Total params: 306,698
Trainable params: 306,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
118/118 [==============================] - 8s 66ms/step - loss: 2.0920 - accuracy: 0.2102 - val_loss: 1.5840 - val_accuracy: 0.2844
Epoch 2/50
118/118 [==============================] - 8s 64ms/step - loss: 1.4557 - accuracy: 0.3492 - val_loss: 1.4235 - val_accuracy: 0.3768
Epoch 3/50
118/118 [==============================] - 8s 64ms/step - loss: 1.3521 - accuracy

* 50 iter的結果比10 iter好，表現最好的iter為46, with 87.6%的Accuracy  

In [21]:
# Dropout, 50 iter
lstm_main(50)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 256)               304128    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                2570      
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
Total params: 306,698
Trainable params: 306,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
118/118 [==============================] - 12s 103ms/step - loss: 2.1430 - accuracy: 0.2011 - val_loss: 1.6572 - val_accuracy: 0.2384
Epoch 2/50
118/118 [==============================] - 12s 99ms/step - loss: 1.6193 - accuracy: 0.2956 - val_loss: 1.4546 - val_accuracy: 0.3618
Epoch 3/50
118/118 [==============================] - 11s 97ms/step - loss: 1.5312 - acc

* 加入Dropout層的結果沒有比較好，最佳在49 iter, with 80.3% Accuracy。  
* 原本的LSTM可能有overfitting的問題：  
  - 然因Eur/ Usd算穩定，若沒有劇烈匯率變化，在預測未來資料的Candlestick型態上便沒有問題。  
  - 若匯率有較大變化，便應該先按時間前後切資料，再進行Shuffle，以進一步驗證是否有overfitting。

#### 3. Use CNN model to classify customized candlestick pattern
* candlestick_model_cnn.py

In [6]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle

from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPool2D


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def get_cnn_model(params):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(10, 10, 4)))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    return model

def train_model(params, data):
    model = get_cnn_model(params)
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],
                     batch_size=params['batch_size'], epochs=params['epochs'], verbose=2)
    return (model, hist)

def print_result(data, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(data['train_gaf'])
    test_pred = model.predict_classes(data['test_gaf'])
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

In [7]:
PARAMS = {}
PARAMS['pkl_name'] = '/content/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl'
PARAMS['classes'] = 9
PARAMS['lr'] = 0.01
PARAMS['epochs'] = 10
PARAMS['batch_size'] = 64
PARAMS['optimizer'] = optimizers.SGD(lr=PARAMS['lr'])

# ---------------------------------------------------------
# load data & keras model
data = load_pkl(PARAMS['pkl_name'])
# train cnn model
model, hist = train_model(PARAMS, data)
# train & test result
scores = model.evaluate(data['test_gaf'], data['test_label_arr'], verbose=0)
print('CNN test accuracy:', scores[1])
print_result(data, model)

Epoch 1/10
235/235 - 6s - loss: 1.5722 - accuracy: 0.4136
Epoch 2/10
235/235 - 6s - loss: 0.7776 - accuracy: 0.7171
Epoch 3/10
235/235 - 6s - loss: 0.6014 - accuracy: 0.7831
Epoch 4/10
235/235 - 6s - loss: 0.5318 - accuracy: 0.8064
Epoch 5/10
235/235 - 6s - loss: 0.4928 - accuracy: 0.8212
Epoch 6/10
235/235 - 6s - loss: 0.4662 - accuracy: 0.8313
Epoch 7/10
235/235 - 6s - loss: 0.4415 - accuracy: 0.8409
Epoch 8/10
235/235 - 6s - loss: 0.4303 - accuracy: 0.8434
Epoch 9/10
235/235 - 6s - loss: 0.4114 - accuracy: 0.8495
Epoch 10/10
235/235 - 6s - loss: 0.4002 - accuracy: 0.8551
CNN test accuracy: 0.8575999736785889
[[1954  138  149  130  194  107  187   87   54]
 [  12 1485    0    1    0    0    2    0    0]
 [  22    0 1452    0   26    0    0    0    0]
 [  17   40    0 1412    0    0    0   31    0]
 [   8    0   17    0 1464    0    1    0   10]
 [  62    2    0    2    0 1397    4   33    0]
 [  23    2    5    0    3    0 1451    0   16]
 [  30    6    0  244    0    8    0 1212    

* 在10個iter下，CNN的預測能力明顯好於預測Sequential的LSTM。
* 如同MNIST資料集，因CandleStick應該也是電腦看得懂的視覺資料。

## Reference